## Create Cluster
To install, first create a k3d / kind cluster
```k3d cluster create kubecon-crossplane```

In [31]:
!k3d cluster delete kubecon-crossplane

INFO[0000] Deleting cluster 'kubecon-crossplane'        
INFO[0001] Deleting cluster network 'k3d-kubecon-crossplane' 
INFO[0002] Deleting 1 attached volumes...               
INFO[0002] Removing cluster details from default kubeconfig... 
INFO[0002] Removing standalone kubeconfig file (if there is one)... 
INFO[0002] Successfully deleted cluster kubecon-crossplane! 


In [32]:
!k3d cluster create kubecon-crossplane

INFO[0000] Prep: Network                                
INFO[0000] Created network 'k3d-kubecon-crossplane'     
INFO[0000] Created image volume k3d-kubecon-crossplane-images 
INFO[0000] Starting new tools node...                   
INFO[0000] Starting node 'k3d-kubecon-crossplane-tools' 
INFO[0001] Creating node 'k3d-kubecon-crossplane-server-0' 
INFO[0001] Creating LoadBalancer 'k3d-kubecon-crossplane-serverlb' 
INFO[0001] Using the k3d-tools node to gather environment information 
INFO[0001] HostIP: using network gateway 172.18.0.1 address 
INFO[0001] Starting cluster 'kubecon-crossplane'        
INFO[0001] Starting servers...                          
INFO[0001] Starting node 'k3d-kubecon-crossplane-server-0' 
INFO[0004] All agents already running.                  
INFO[0004] Starting helpers...                          
INFO[0004] Starting node 'k3d-kubecon-crossplane-serverlb' 
INFO[0010] Injecting records for hostAliases (incl. host.k3d.internal) and for 2 network members into

## Get Credentials

In [33]:
!atmos use -e scratch-oamdemo | grep AWS > .aws-creds

# Note: As part of diagnostics info, we capture the username and the ip address of the machine from where the cli is being run.
# Info: Welcome to Atmos CLI. The latest version is 3.10.0


## Install Addons

We need a few prerequisites on the cluster to be able to run the demo:
- demo-resources (addon)
- chartmuseum (addon)
- vela-workflow (addon)

The install script can be run without the workflow (to demo standalone WorkflowRun installations), or it can be run together (quicker for the demo - setup in advance).

In [34]:
!./install.sh --deploy-workflow

🚀 Installing Kubecon Demo...
✅ Found .aws-creds file
📦 KubeVela is not running. Installing KubeVela...
Check Requirements ...
Installing KubeVela Core ...
Helm Chart used for KubeVela control plane installation: https://kubevela.github.io/charts/vela-core-1.10.4.tgz 
Start upgrading Helm Chart kubevela in namespace vela-system
getting history for release kubevela
creating 1 resource(s)
CRD applicationrevisions.core.oam.dev is already present. Skipping.
creating 1 resource(s)
CRD applications.core.oam.dev is already present. Skipping.
creating 1 resource(s)
CRD componentdefinitions.core.oam.dev is already present. Skipping.
creating 1 resource(s)
CRD definitionrevisions.core.oam.dev is already present. Skipping.
creating 1 resource(s)
CRD policies.core.oam.dev is already present. Skipping.
creating 1 resource(s)
CRD policydefinitions.core.oam.dev is already present. Skipping.
creating 1 resource(s)
CRD resourcetrackers.core.oam.dev is already present. Skipping.
creating 1 resource(s)
CR

## What's Running?

### What is running in the cluster?
 
- The chartmuseum Addon which will be used to host our custom addons (Crossplane etc.) 
- The vela-workflow Addon (available from the KubeVela addon repository) which adds standalone Workflow support
- The kube-trigger Addon (available from the KubeVela addon repository) which installs kube-trigger to enable programmable event-driven actions
- The FluxCD Addon available from the KubeVela addon repository
- The [Crossplane Addon](./addons/crossplane/) which installs the Crossplane Helm Chart and adds a Provider definition
- The [Crossplane AWS Addon](./addons/crossplane-aws/) which installs:
  - The aws-provider-family and aws-s3 Crossplane Providers
  - The default Crossplane ProviderConfig
  - S3 ComponentDefinition and associated TraitDefinitions (Encryption, Lifecycle etc.)
- The [Tenant](./addons/tenant/) which installs:
  - A "tenant" configuration template
  - A TriggerService to monitor for instances of the tenant configuration
  - Create + Delete Application actions which will build the tenant app (creates a namespace + S3 bucket for each configured tenant)

In [35]:
## Verify what is running
!vela list --all-namespaces

NAMESPACE  	APP                 	COMPONENT                          	TYPE               	TRAITS                        	PHASE  	HEALTHY	STATUS                                                      	CREATED-TIME                 
vela-system	addon-chartmuseum   	chartmuseum                        	webservice         	storage,env                   	running	healthy	Ready:1/1                                                   	2025-11-11 11:52:37 +0000 GMT
vela-system	└─                  	gcs-credential                     	k8s-objects        	                              	running	healthy	                                                            	2025-11-11 11:52:37 +0000 GMT
vela-system	addon-crossplane    	crossplane                         	helm               	                              	running	healthy	Fetch repository successfully, Create helm release          	2025-11-11 11:55:26 +0000 GMT
           	                    	                                   	                   	   

In [36]:
# Use the addon CLI for another view of addons and their versions
!vela addon list | grep enabled

chartmuseum              	KubeVela	ChartMuseum is an open-source and easy to deploy Helm Chart ...	[4.2.1, 4.2.0, 4.1.0]        	enabled (4.2.1) 
kube-trigger             	KubeVela	kube-trigger watches events and triggers actions.              	[1.0.0]                      	enabled (1.0.0) 
velaux                   	KubeVela	KubeVela User Experience (UX). An extensible, application-or...	[v1.9.4, v1.9.3, v1.9.2, ...]	enabled (v1.9.4)
vela-workflow            	KubeVela	vela-workflow provides the capability to run a standalone wo...	[v0.6.3, v0.6.2, v0.6.1, ...]	enabled (v0.6.3)
fluxcd                   	KubeVela	Extended workload to do continuous and progressive delivery    	[3.0.1, 3.0.0, 2.4.0, ...]   	enabled (3.0.1) 


In [37]:
## Crossplane Definitions
!vela def list | grep crossplane

crossplane-provider          	ComponentDefinition   	vela-system	crossplane    	Crossplane Provider                                         
s3-bucket                    	ComponentDefinition   	vela-system	crossplane-aws	AWS S3 Bucket managed by Crossplane                         
s3-object                    	ComponentDefinition   	vela-system	crossplane-aws	AWS S3 Object managed by Crossplane                         
s3-encryption                	TraitDefinition       	vela-system	crossplane-aws	Configure server-side encryption for S3 bucket              
s3-lifecycle                 	TraitDefinition       	vela-system	crossplane-aws	Configure lifecycle rules for S3 bucket                     
s3-public-access-block       	TraitDefinition       	vela-system	crossplane-aws	Configure public access blocking for S3 bucket              
s3-tags                      	TraitDefinition       	vela-system	crossplane-aws	Add custom tags to S3 bucket                                
s3-versioning

### How was all of this installed? 

- We used a KubeVela [WorkflowRun](./workflows/install.yaml) to install Addons in an orchestrated manner.

## Demo

#### Start VelaUX

Run a port-forward to VelaUX **in the background**.

In another terminal window, run:
```
vela port-forward addon-velaux -n vela-system
```

**NOTE**: If you encounter issues with the site not loading, change `127.0.0.1` to `localhost` in the browser window. 

#### Deploy the Application

This will deploy the [Webservice Application](./apps/webservice-s3-app.yaml) which provisions an S3 bucket using Crossplane and deploys a Python webservice that interacts with it.

In [58]:
## Install the Webservice App
!vela up -f ./apps/webservice-s3-app.yaml

Applying an application in vela K8s object format...
✅ App has been deployed 🚀🚀🚀
    Port forward: vela port-forward webservice-s3-demo
             SSH: vela exec webservice-s3-demo
         Logging: vela logs webservice-s3-demo
      App status: vela status webservice-s3-demo
        Endpoint: vela status webservice-s3-demo --endpoint
Application default/webservice-s3-demo applied.


#### Get the applications status
- The app should not be marked healthy until the S3 bucket is successfully synced.
- Once synced the application status will reflect the S3s ARN etc.
- The output may be truncated here - open in a text editor to see the full output.

**Note**: The S3 health and status logic can be viewed at: [S3 Component](./addons/crossplane-aws/definitions/s3-bucket-base.cue)

In [44]:
## Follow it's status - keep running until it's Healthy (Details: running)
!vela status webservice-s3-demo -n default

About:

  Name:      	webservice-s3-demo           
  Namespace: 	default                      
  Created at:	2025-11-11 11:58:21 +0000 GMT
  Healthy:   	✅                           
  Details:   	running                      

Workflow:

  mode: StepByStep-DAG
  finished: true
  Suspend: false
  Terminated: false
  Steps
  - id: 7g28l5kpxh
    name: create-s3-bucket
    type: apply-component
    phase: succeeded 
  - id: 5a7yumtltr
    name: create-welcome-file
    type: apply-component
    phase: succeeded 
  - id: hqljm9pj9d
    name: create-demo-data
    type: apply-component
    phase: succeeded 
  - id: 1jd2w2k185
    name: create-readme-file
    type: apply-component
    phase: succeeded 
  - id: 4ghrebd8si
    name: create-webservice
    type: apply-component
    phase: succeeded 

Services:

  - Name: demo-webservice  
    Cluster: local
    Namespace: default
    Type: webservice
    Health: ✅ 
      Message: Ready:2/2
    Traits:
      Type: scaler
      Health: ✅ 
      Typ

#### Port Forward to the Front End

Setup a port forward to the application to see how it is retrieving data from the S3 bucket that was just provisioned. 

In [49]:
# Vela port-forward command
!vela port-forward webservice-s3-demo -n default 5010:5000

trying to connect the remote endpoint svc/demo-webservice 5010:5000 ..Forwarding from 127.0.0.1:5010 -> 5000
Forwarding from [::1]:5010 -> 5000

Forward successfully! Opening browser ...
Handling connection for 5010
Handling connection for 5010


OSError: [Errno 5] Input/output error

In [46]:
## Get the application tree view
!vela status webservice-s3-demo --tree  

CLUSTER       NAMESPACE     RESOURCE                                 STATUS    
local     ─┬─ -         ─┬─ Bucket/webservice-s3-demo-demo-s3-bucket updated   
           │             ├─ Object/webservice-s3-demo-demo-data      updated   
           │             ├─ Object/webservice-s3-demo-readme-file    updated   
           │             └─ Object/webservice-s3-demo-welcome-file   updated   
           └─ default   ─┬─ Service/demo-webservice                  updated   
                         └─ Deployment/demo-webservice               updated   


#### View in VelaUX

View it in the VelaUX UI (port forward earlier).

- [Workflow](http://localhost:8000/applications/webservice-s3-demo/envbinding/default/workflow/records/webservice-s3-demo-v1)
- [Status] (http://localhost:8000/applications/webservice-s3-demo/envbinding/default/status)
- [Logs] (http://localhost:8000/applications/webservice-s3-demo/envbinding/default/logs)

Login Details:
- username: admin
- password: VelaUX12345

#### Delete the Application

Delete the application - which will remove the webservice and bucket.

In [ ]:
# Delete the Application
vela delete webservice-s3-demo -n default --yes

## Demo 2 (Optional) - kube-trigger
This is a more advanced demo that shows off KubeVela ConfigTemplate & kube-triggers capabilities. 

In [ ]:
# Create a kubecon tenant config
!vela config create kubecon-tenant --template=tenant name=kubecon

The instance of the ConfigTemplate will create a ConfigMap on the cluster according to the ConfigTemplate specification.

In [61]:
# Get the ConfigMap created by the ConfigTemplate
!kubectl get configmap tenant-kubecon-tenant -n vela-system 

NAME                    DATA   AGE
tenant-kubecon-tenant   2      2m7s


In [63]:
## Describe the ConfigMap   
!kubectl get configmap tenant-kubecon-tenant -n vela-system -o yaml

apiVersion: v1
data:
  name: kubecon
  namespace: kubecon
kind: ConfigMap
metadata:
  creationTimestamp: "2025-11-11T12:15:52Z"
  labels:
    config.oam.dev/config-type: tenant
    config.oam.dev/tenant: kubecon
    config.oam.dev/tenant-namespace: kubecon
    oam.dev/render-hash: d4b75158d963b077
  name: tenant-kubecon-tenant
  namespace: vela-system
  resourceVersion: "8033"
  uid: 286cc315-4102-4e87-8cac-e345462cb871


The [TriggerService](./addons/tenant/resources/trigger-service.cue) setup in the tenant addon watches for ConfigMaps with label `config.oam.dev/config-type: tenant`.

This routes to the [create-app](./addons/tenant/resources/create-app.cue) action, which creates an Application for the configured tenant.

When deleted, it routes to the [delete-app](./addons/tenant/resources/delete-app.cue) action.

#### View the Generated Application

In [64]:
!vela status tenant-kubecon -n vela-system

About:

  Name:      	tenant-kubecon               
  Namespace: 	vela-system                  
  Created at:	2025-11-11 12:15:52 +0000 GMT
  Healthy:   	✅                           
  Details:   	running                      

Workflow:

  mode: StepByStep-DAG
  finished: true
  Suspend: false
  Terminated: false
  Steps
  - id: mbw151k7an
    name: create-namespace
    type: apply-component
    phase: succeeded 
  - id: uvy4dem9yh
    name: create-s3-bucket
    type: apply-component
    phase: succeeded 

Services:

  - Name: tenant-s3-bucket  
    Cluster: local
    Namespace: vela-system
    Type: s3-bucket
    Health: ✅ 
      Message: Synced: arn:aws:s3:::tenant-kubecon-tenant-s3-bucket
      Status Details: 
        bucketDomainName: tenant-kubecon-tenant-s3-bucket.s3.amazonaws.com
        bucketName: tenant-kubecon-tenant-s3-bucket
        syncReason: Available
        syncStatus: Ready
        bucketArn: arn:aws:s3:::tenant-kubecon-tenant-s3-bucket
        bucketRegionalDomain

#### View it in VelaUX

- [Workflow] (http://localhost:8000/applications/tenant-kubecon/envbinding/system/workflow/records/tenant-kubecon-v1)
- [Status](http://localhost:8000/applications/tenant-kubecon/envbinding/system/status)
- [Tenant Config](http://localhost:8000/configs/tenant/config)

#### Delete the Config

In [67]:
# Delete the config to trigger tenant deletion
!vela config delete kubecon-tenant --yes

the config kubecon-tenant deleted successfully


In [71]:
# Monitor the app status - it will go into status deleting and then disappear
!vela status tenant-kubecon -n vela-system

Error: failed to load application tenant-kubecon from namespace vela-system: applications.core.oam.dev "tenant-kubecon" not found
